# Fashion MNIST Image Classification - Azure ML SDK Deployment

In this Fashion MNIST notebook we introduce how to take the models you have registered and versioned, deploy them to production and monitor them 

This code will show how Azure ML SDK can support your machine learning project with:
* creating a training and scoring script for your model
* creating a bespoke environment for your model to run in (libraries and packages to install)
* setting up a container service to host your model
* deploying your model to the cloud container service
* retrieving the service scoring URL for your hosted model
* sending test data to your hosted model to be scored and returned to compare for the accuracy of your model on your validation set

## Import packages 

In [1]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import azureml
from azureml.core import Workspace, Run

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  0.1.65


## Load workspace and download registered model 

In [2]:
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model=Model(ws, 'keras_dl_fashion_test')
#model.download(target_dir = '.')
import os 
# verify the downloaded model file
os.stat('model.h5')

Found the config file in: C:\Users\amynic\Documents\CloudDeveloperAdvocate\Deeplearning\ai-world-summit\config.json


os.stat_result(st_mode=33206, st_ino=3096224744218411, st_dev=2934966739, st_nlink=1, st_uid=0, st_gid=0, st_size=2694024, st_atime=1538936276, st_mtime=1538936279, st_ctime=1538936276)

## Load and format Test dataset

In [3]:
import tensorflow as tf
import os
import time
os.environ["TF_CPP_MIN_LOG_LEVEL"]= "2"
print("tensorflow Version is: " + str(tf.__version__))

import numpy as np
os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras import backend as K
print(os.environ['KERAS_BACKEND'])

#Fashion MNIST Dataset CNN model development: https://github.com/zalandoresearch/fashion-mnist
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import utils, losses, optimizers
import matplotlib.pyplot as plt
from utils import load_data

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

#no. of classes
num_classes = 10
# Define the text labels
fashion_mnist_labels = ["Top",          # index 0
                        "Trouser",      # index 1
                        "Jumper",       # index 2 
                        "Dress",        # index 3 
                        "Coat",         # index 4
                        "Sandal",       # index 5
                        "Shirt",        # index 6 
                        "Trainer",      # index 7 
                        "Bag",          # index 8 
                        "Ankle boot"]   # index 9

x_test = x_test.astype('float32')
x_test /= 255
y_test = utils.to_categorical(y_test,  num_classes)

#X_test = load_data('./data/test-images.gz', False) / 255.0
#y_test = load_data('./data/test-labels.gz', True).reshape(-1)

tensorflow Version is: 1.11.0
tensorflow


Using TensorFlow backend.


## Create a score.py script

In [4]:
%%writefile score.py
import keras as K
from azureml.core.model import Model
from keras.models import Sequential
from keras.models import load_model
import h5py
import sys
import os
import numpy as np
import json

def init():
    global model
    print("Python version: " + str(sys.version) + ", keras version: " + K.__version__)
    print("Executing init() method...")
    model_path = Model.get_model_path('model.h5')
    print("got model...")
    model = K.models.load_model(model_path)
    print("loaded model...")

def run(raw_data):
    print("Executing run(raw_data) method...")
    data = np.array(json.loads(raw_data)['data'])
    data = np.reshape(data, (30,28,28,1))
    y_hat = model.predict(data)
    print("Eexcuted predictions...")
    return json.dumps(y_hat.tolist())

Overwriting score.py


## Create a bespoke environment to run your model in 

In [5]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
myenv.add_conda_package("keras")
myenv.add_tensorflow_pip_package(core_type='cpu', version='1.11.0')

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [6]:
with open("myenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults
  - tensorflow==1.11.0
- scikit-learn
- keras



## Setup a Container service to host your model in

In [7]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "fashion-MNIST",  "method" : "keras"}, 
                                               description='Deep Learning to classify fashion clothing items, using Keras')

In [9]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  description = "Deep Learning to classify fashion clothing items, using Keras",
                                                  tags = {"data": "fashionmnist", "type": "classification"}
                                                 )

service_name = 'deployfashion-test-2'
service = Webservice.deploy(deployment_config = aciconfig,
                                image_config = image_config,
                                model_paths = ['model.h5'],
                                name = service_name,
                                workspace = ws)

service.wait_for_deployment(show_output = True)
print(service.state)

Registering model model.h5
Creating image
Image creation operation finished for image deployfashion-test-2:1, operation "Succeeded"
Creating service
Running.......................................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy
Wall time: 9min 49s


In [10]:
print(service.scoring_uri)

http://40.114.204.17:80/score


## Test your model by calling the hosted API with test data

In [12]:
import json
s = 30
e = s + 30

#sample_indices = (x_test[0:n])

test_samples = json.dumps({"data": x_test[s:e].tolist()})
test_samples = bytes(test_samples, encoding = 'utf8')

test_labels = y_test[s:e]

##testing input correct
body = np.array(json.loads(test_samples)['data'])

# predict using the deployed model
result = json.loads(service.run(input_data=test_samples))

for i in range(0, len(result)):
    indice_prediction = np.argmax(result[i])
    indice_label = np.argmax(test_labels[i])
    if indice_prediction == indice_label:
        print("correct prediction for sample " + str(i))
    else:
        print("INCORRECT prediction for sample" + str(i))

correct prediction for sample 0
correct prediction for sample 1
correct prediction for sample 2
correct prediction for sample 3
correct prediction for sample 4
correct prediction for sample 5
correct prediction for sample 6
correct prediction for sample 7
correct prediction for sample 8
correct prediction for sample 9
correct prediction for sample 10
correct prediction for sample 11
INCORRECT prediction for sample12
correct prediction for sample 13
correct prediction for sample 14
correct prediction for sample 15
correct prediction for sample 16
correct prediction for sample 17
correct prediction for sample 18
INCORRECT prediction for sample19
correct prediction for sample 20
correct prediction for sample 21
correct prediction for sample 22
correct prediction for sample 23
correct prediction for sample 24
correct prediction for sample 25
correct prediction for sample 26
correct prediction for sample 27
correct prediction for sample 28
correct prediction for sample 29
